# **데이터셋 합치기**

## **계층적 인덱싱(Hierarchical Indexing)**

지금까지 1차원 및 2차원 데이터를 각각 시리즈와 데이터프레임에 저장하였다. 종종 고차원 데이터 즉 두 개 이상의 키로 인덱싱된 데이터를 저장하는 것이 필요할 때가 있다.이렇 때는 **계층적 인덱싱(Hierarchical Indexing)** 혹은 **다중 인덱싱(multi-indexing)**을 사용한다.

In [ ]:
import pandas as pd
import numpy as np

### 다중 인덱스 생성 방법

다중 인덱스를 가진 시리즈 또는 데이터프레임을 생성하는 가장 간단한 방법은 생성자에 두 개 이상의 인덱스 목록을 전달하는 것이다. 예를 들어 서로 다른 년도(years)와 주(state)의 인구 데이터를 다음과 같이 다중 인덱스를 가진 시리즈로 저장할 수 있다.

In [ ]:
pop = pd.Series([33871648, 37253956, 18976457, 19378102, 20851820, 25145561],
                index=[['California', 'California', 'New York', 'New York', 'Texas', 'Texas'],
                        [2000, 2010, 2000, 2010, 2000, 2010]])
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

전체 인구와 미성년 인구를 저장하는 데이터프레임을 만들어보자.

In [ ]:
pop_df = pd.DataFrame({'total': [33871648, 37253956, 18976457, 19378102, 20851820, 25145561],
                       'under18': [9267089, 9284094, 4687374, 4318033, 5906301, 6879014]},
                       index=[['California', 'California', 'New York', 'New York', 'Texas', 'Texas'],
                              [2000, 2010, 2000, 2010, 2000, 2010]]
                       )
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

멀티 인덱스를 명시적으로 생성하는 것이 유용한 경우가 있는데, 여기서는 이러한 방법 중 몇 가지를 살펴보자.

각 레벨 내의 인덱스 값을 제공하는 간단한 배열 목록에서 `MultiIndex`를 구성할 수 있다:

In [ ]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

혹은 튜플 리스트로부터 구성할 수도 있다:

In [ ]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

단일 인덱스의 곱으로 구성할 수도 있다:

In [ ]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

이렇게 생성된 `MultiIndex` 객체는 `Series` 또는 `DataFrame`을 생성할 때 인덱스 인수로 전달할 수 있다.

### `MultiIndex` 레벨 이름

때때로 멀티 인덱스의 레벨 이름을 지정하는 것이 편리할 때가 있다. 이는 위의 `MultiIndex` 생성자 중 하나에 `names` 인수를 전달하거나 인덱스의 `names` 속성을 사후에 설정하여 수행할 수 있다:

In [ ]:
pop_df.index.names = ['state', 'year']
pop_df

total  under18
state      year                   
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

### 열의 `MultiIndex`

데이터 프레임에서 행과 열은 완전히 대칭이며, 행에 여러 수준의 인덱스가 있을 수 있는 것처럼 열에도 여러 수준의 인덱스가 있을 수 있다. 다음은 행과 열 모두에 대한 다중 인덱싱이 유용하게 사용될 수 있는 한 예이다.

In [ ]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      42.0  36.0  68.0  39.3  43.0  35.7
     2      33.0  37.9  27.0  37.0  30.0  37.0
2014 1      33.0  38.1  41.0  36.6  52.0  38.1
     2      38.0  34.6  26.0  36.0  29.0  37.0

이 데이터는 기본적으로 4차원 데이터이며, 여기서 차원은 환자(subject), 측정값 유형(type), 연도(year) 및 방문 횟수(visit)이다.

### 멀티인덱스 인덱싱 및 슬라이싱

멀티 인덱스의 인덱싱 및 슬라이싱은 직관적으로 설계되었으며, 인덱스를 추가된 차원으로 생각하면 도움이 된다.

#### `Series`의 멀티인덱스

앞서 살펴본 주 인구의 멀티인덱싱된 `Series`를 생각해 보자:

In [ ]:
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [ ]:
pop.loc['California', 2000]

33871648

In [ ]:
pop.loc['California']

2000    33871648
2010    37253956
dtype: int64

In [ ]:
pop.loc['California':'New York']

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [ ]:
pop.loc[:, 2000]

California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [ ]:
pop.loc[pop > 22000000]

California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [ ]:
pop.loc[['California', 'Texas']]

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

#### `DataFrame`의 멀티인덱스

In [ ]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      43.0  36.7  35.0  36.0  25.0  37.1
     2      44.0  38.8  43.0  36.9  32.0  35.6
2014 1      40.0  36.3  29.0  36.7  29.0  36.6
     2      32.0  35.8  21.0  36.8  36.0  35.6

In [ ]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      35.0  36.0
     2      43.0  36.9
2014 1      29.0  36.7
     2      21.0  36.8

`DataFrame`에서 다중 인덱스로 인덱스하면 열(column)에 적용됨을 기억하라.

In [ ]:
health_data['Guido', 'HR']

year  visit
2013  1        35.0
      2        43.0
2014  1        29.0
      2        21.0
Name: (Guido, HR), dtype: float64

또한 단일 인덱스의 경우와 마찬가지로 `loc`과 `iloc` 인덱서를 사용할 수 있다.

In [ ]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      43.0  36.7
     2      44.0  38.8

이러한 인덱서는 기본 2차원 데이터의 배열과 같은 보기를 제공하지만, `loc` 또는 `iloc`의 각 개별 인덱스에는 여러 인덱스의 튜플을 전달할 수 있다. 예를 들어

In [ ]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        43.0
      2        44.0
2014  1        40.0
      2        32.0
Name: (Bob, HR), dtype: float64

In [ ]:
health_data.loc[(2014, 2), ('Bob', 'HR')]

38.0

다중 인덱싱된 시리즈 및 데이터프레임에서 데이터와 상호 작용하는 방법은 매우 다양하며, 이 도구에 익숙해지는 가장 좋은 방법은 직접 사용해 보는 것이다!

### 인덱스의 Stacking과 Unstacking

앞서 간략하게 살펴본 것처럼, stacking된 다중 인덱스에서 간단한 2차원 표현으로 데이터 집합을 변환할 수 있으며, 사용할 수준을 선택적으로 지정할 수 있다:

In [ ]:
pop.unstack(level=0)

,California,New York,Texas
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [ ]:
pop.unstack(level=1)

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


`unstack()`의 반대 연산은 `stack()`이며 원래 시리즈를 복구하는 데 사용할 수 있다:

In [ ]:
pop.unstack().stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

## **데이터셋 연결 하기: `Concat`**

서로 다른 데이터 원본을 결합하는 것에는 두 개의 서로 다른 데이터셋을 연결하는 매우 간단한 연결(concatenate)부터 데이터 집합 간의 중복을 올바르게 처리하는 보다 복잡한 데이터베이스 스타일의 조인(join) 및 병합(merge)까지 모든 것이 포함될 수 있다.
시리즈와 데이터프레임은 이러한 유형의 연산을 염두에 두고 만들어졌으며, Pandas에는 이러한 종류의 데이터 랭글링을 빠르고 간단하게 만들어주는 함수와 메서드가 포함되어 있다.

여기서는 ``pd.concat`` 함수를 사용하여 ``Series``와 ``DataFrame``을 연결하는 간단한 경우에 대해 살펴보고, 나중에 Pandas에서 구현된 보다 정교한 인메모리 병합(merge) 및 조인(join)에 대해 자세히 알아보겠다.

편의상 아래에서 유용한 특정 형식의 데이터 프레임을 생성하는 함수를 정의하겠다:

In [ ]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


또한 여러 개의 데이터프레임을 나란히 표시할 수 있는 퀵 클래스를 만들어 보겠다. 이 코드는 IPython이 객체를 디스플레이하는 데 사용하는 특수한 _repr_html_ 메서드를 사용한다:

In [ ]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args

    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)

    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

### NumPy 배열의 연결

시리즈 및 데이터프레임 객체의 연결(concatenation)은 `np.concatenate` 함수를 통해 수행할 수 있는 Numpy 배열의 연결과 매우 유사하다. 이 함수를 사용하면 두 개 이상의 배열의 내용을 하나의 배열로 결합할 수 있다:

In [ ]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

첫 번째 인수는 연결할 배열의 목록 또는 튜플이다. 또한 `axis` 인자를 총해서 결과를 연결할 축을 지정할 수 있다:

In [ ]:
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

### ``pd.concat``을 사용한 간단한 연결

Pandas에는 `pd.concat()` 함수가 있는데, 이 함수는 `np.concatenate`와 비슷하지만 잠시 후에 설명할 여러 가지 옵션을 포함하고 있다.
```python
# Signature in Pandas v1.5.3
pandas.concat(objs, *, axis=0, join='outer', ignore_index=False, keys=None,
              levels=None, names=None, verify_integrity=False, sort=False, copy=True)
```

배열의 간단한 연결에 `np.concatenate()`를 사용하는 것과 마찬가지로 시리즈 또는 데이터프레임 객체의 간단한 연결에는 `pd.concat()`을 사용할 수 있다:

In [ ]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

또한 데이터프레임과 같은 고차원 객체를 연결할 때도 작동한다:

In [ ]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
display('df1', 'df2', 'pd.concat([df1, df2])')

df1
    A   B
1  A1  B1
2  A2  B2

df2
    B   C
2  B2  C2
4  B4  C4

pd.concat([df1, df2])
     A   B    C
1   A1  B1  NaN
2   A2  B2  NaN
2  NaN  B2   C2
4  NaN  B4   C4

기본적으로 연결은 데이터 프레임 내에서 행 단위로 이루어진다(즉, `axis=0`). `np.concatenate`와 마찬가지로 `pd.concat`에서도 연결이 수행될 축을 지정할 수 있다.

In [ ]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
display('df3', 'df4', "pd.concat([df3, df4], axis='columns')")

df3
    A   B
0  A0  B0
1  A1  B1

df4
    C   D
0  C0  D0
1  C1  D1

pd.concat([df3, df4], axis='columns')
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1

`axis=1`로 지정할 수도 있지만 여기서는 보다 직관적인 `axis='columns'`을 사용했다.

### 중복된 인덱스(Duplicate indices)

`np.concatenate`와 `pd.concat`의 한 가지 중요한 차이점은 `pd.concat`에서는 결과에 중복 인덱스가 있더라도 인덱스를 보존한다는 것이다. 간단한 예제를 살펴보자:


In [ ]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # make duplicate indices!
display('x', 'y', 'pd.concat([x, y])')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


결과에서 반복되는 인덱스를 볼 수 있다. 이는 때론 유용하지만 그렇지 않은 경우도 많다. `pd.concat()`은 이를 처리할 수 있는 몇 가지 방법을 제공힌다.

#### 반복을 오류로 처리하기

`pd.concat()` 결과의 인덱스가 겹치지 않는지 간단히 확인하려는 경우 `verify_integrity` 플래그를 지정할 수 있다. 이 플래그를 `True`로 설정하면 중복 인덱스가 있는 경우 연결 시 예외가 발생한다. 다음은 명확성을 위해 오류 메시지를 포착하여 인쇄하는 예제이다:

In [ ]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


#### 인덱스 무시하기

때로는 인덱스 자체가 중요하지 않아서 단순히 무시하고 싶을 때가 있다. 이 옵션은 `ignore_index` 플래그를 사용하여 지정할 수 있다. 이 옵션을 `True`로 설정하면 결과 시리즈에 대해 새로운 정수 인덱스를 생성한다:

In [ ]:
display('x', 'y', 'pd.concat([x, y], ignore_index=True)')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


#### 다중 인덱스를 추가하기

또 다른 방법은 ``keys`` 옵션을 사용하여 데이터 소스에 대한 레이블을 지정하는 것이다. 그러면 계층적으로 인덱싱된 시리즈가 생성된다:

In [ ]:
display('x', 'y', "pd.concat([x, y], keys=['x', 'y'])")

x
    A   B
0  A0  B0
1  A1  B1

y
    A   B
0  A2  B2
1  A3  B3

pd.concat([x, y], keys=['x', 'y'])
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

## **데이터셋 병합하기: merge 및 join**

Pandas가 제공하는 필수 기능 중 하나는 효율적인 조인(join) 및 병합(merge) 작업이다. 이를 위한 주요 인터페이스는 `pd.merge` 함수이며, 이것이 실제로 어떻게 작동하는지에 대한 몇 가지 예를 살펴보겠다.

### 병합의 유형

`pd.merge()`함수는 일대일(one-to-one), 다대일(many-to-one) 및 다대다(many-to-many) 병합 등 여러 유형을 구현한다.
세 가지 유형은 모두 `pd.merge()` 인터페이스에 대한 동일한 호출을 통해 액세스되며, 수행되는 병합 유형은 입력 데이터의 형식에 따라 다르다.
여기에서는 먼저 세 가지 유형의 병합에 대한 간단한 예제를 보고 자세한 옵션에 대해서는 아래에서 자세히 설명한다.


#### 일대일(one-to-one) 병합

가장 간단한 유형은 일대일 병합으로 연결(concatenation)과 여러 면에서 매우 유사하다. 구체적인 예로, 한 회사의 여러 직원에 대한 정보가 포함된 다음 두 데이터 프레임을 살펴보자:


In [ ]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue', 'Kim'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR', 'CEO']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue', 'Lee'],
                    'hire_date': [2004, 2008, 2012, 2014, 2016]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
4,Kim,CEO
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [ ]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


`pd.merge()` 함수는 두 데이터프레임에 `employee` 열이 있음을 인식하고 이 열을 키로 사용하여 조인한다. 병합 결과는 두 데이터프레임의 정보를 결합한 새 데이터 프레임이다. 각 열의 항목 순서가 반드시 유지되는 것은 아니다. 또한 인덱스별로 병합하는 특수한 경우를 제외하고 일반적으로 병합하면 인덱스가 삭제된다는 점에 유의하라라.

#### 다대일(many-to-one) 병합

다대일 병합은 두 키 열 중 하나에 중복 항목이 포함된 경우이다.
결과 데이터 프레임은 이러한 중복 항목을 적절하게 보존한다.

In [ ]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

결과 데이터 프레임에는 `supervisor` 정보가 포함된 추가 열이 있으며, 이 열에서 가령령 `Guido`와 같은 값은 중복될 수 있다.

#### 다대다(many-to-many) 병합

두 데이터프레임의 키 열 모두에 중복이 있는 경우 결과는 다대다 병합이 된다. 예를 들어 특정 그룹(group)과 연관된 하나 이상의 스킬(skill)을 보여주는 데이터프레임이 있다고 가정해 보자. 다대다 병합을 수행하면 모든 개인에에 대해 관련된 스킬을 구할 수 있다:

In [ ]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


### 병합 키의 지정

`pd.merge()`의 기본 동작은은 두 입력 사이에서 일치하는 열 이름을 하나 이상 찾아서 이를 키로 사용하는 것이다. 그러나 종종 열 이름이 일치하지 않을 수 있으며, `pd.merge()`는 이를 처리하기 위한 다양한 옵션을 제공한다.

#### ``on`` 키워드

가장 간단하게는 병합할 열 이름 또는 열 이름 목록을 ``on`` 키워드로 명시적으로 지정할 수 있다:

In [ ]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2, on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

이 옵션은 양쪽 데이터프레임 모두에 지정된 열 이름이 있는 경우에만 작동한다.

#### ``left_on``과 ``right_on`` 키워드

열 이름이 다른 두 데이터 집합을 병합해야 하는 경우가 있다. 예를 들어, 직원 이름이 'employee'가 아닌 'name'으로 레이블이 지정된 데이터셋이 있을 수 있다. 이 경우 `left_on` 및 `right_on` 키워드를 사용하여 두 열 이름을 지정할 수 있다:

In [ ]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

결과에는 중복된 열이 포함되며, 필요하면 삭제할 수 있다(예: ``DataFrame``의 ``drop()`` 메서드 사용).

In [ ]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


#### ``left_index``와 ``right_index`` 키워드

열로 병합하는 대신 인덱스로 병합하고 싶을 때가 있다. ``left_index`` 및/또는 ``right_index`` 플래그를 지정하여 인덱스를 병합의 키로 사용할 수 있다:

In [ ]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


In [ ]:
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


편의를 위해 ``DataFrame``은 기본적으로 인덱스로 조인하는 병합을 수행하는 ``join()`` 메서드를 제공한다:

In [ ]:
display('df1a', 'df2a', 'df1a.join(df2a)')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


인덱스와 열을 혼합하려는 경우 `left_index`와 `right_on`을 결합하거나 `left_on`와 `right_index`를 결합하여 원하는 동작을 얻을 수 있다:

In [ ]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='name')")

df1a
                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1a, df3, left_index=True, right_on='name')
         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000

### 병합을 위한 집합 연산 지정

앞의 모든 예제에서 병합을 수행할 때 중요한 고려 사항 중 하나인 병합에 사용되는 집합 산술 유형에 대해 자세히 다루지 않았다. 이 문제는 값이 한 키 열에는 나타나지만 다른 키 열에는 나타나지 않을 때 발생한다.

In [ ]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
display('df6', 'df7', 'pd.merge(df6, df7)')

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7)
   name   food drink
0  Mary  bread  wine

여기서는 공통인 이름(name)이 하나만 있는 두 데이터셋을 병합했다. 기본적으로 결과에는 두 입력 집합의 교집합이 포함되며, 이를 내부(inner) 조인이라고 한다. `merge`에서 `how='inner'`로 명시적으로 지정할 수 있다.

In [ ]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


`how` 키워드의 다른 옵션으로는 `'outer'`, `'left'` 및 `'right'`가 있다. 외부(outer) 조인은 입력 열의 합집합에 대한 조인을 반환하고 모든 누락된 값을 `NA`로 채운다:

In [ ]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='outer')
     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer

왼쪽(left) 조인 및 오른쪽(right) 조인은 각각 왼쪽 항목과 오른쪽 항목을 반환한다. 예를 들어:

In [ ]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='left')
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

`how='right'`를 사용하는 것도 비슷한 방식으로 작동한다.
이러한 모든 옵션은 앞의 모든 조인 유형에 바로 적용할 수 있다.

### 중첩된 열 이름:  ``suffixes`` 키워드

두 데이터프레의 열 이름이 충돌되는 경우이다.

In [ ]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
display('df8', 'df9', 'pd.merge(df8, df9, on="name")')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name")
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

출력에 두 개의 충돌하는 열 이름이 있기 때문에 병합 함수는 자동으로 접미사 `_x` 또는 `_y`를 추가하여 출력 열을 고유하게 만든다. 이러한 기본값이 부적절한 경우 `suffixes` 키워드를 사용하여 사용자 지정 접미사를 지정할 수 있다:

In [ ]:
display('df8', 'df9', 'pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])
   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

### 예제: US States Data

병합 및 조인 작업은 서로 다른 원본의 데이터를 결합할 때 가장 자주 발생한다.
여기서는 미국 주와 그 인구에 대한 몇 가지 데이터의 예를 살펴보겠다.
데이터 파일은 [여기](http://github.com/jakevdp/data-USstates/) 에서 찾을 수 있다:

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
data_dir = '/content/drive/MyDrive/DataScience2023/chap04_pandas/datasets/data-USstates'
pop = pd.read_csv(data_dir + '/state-population.csv')
areas = pd.read_csv(data_dir + '/state-areas.csv')
abbrevs = pd.read_csv(data_dir + '/state-abbrevs.csv')

display('pop.head()', 'areas.head()', 'abbrevs.head()')

pop.head()
  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

areas.head()
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

abbrevs.head()
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

이 정보로부터 2010년 인구 밀도에 따라 미국 주의 순위를 매기는 비교적 간단한 일을 하려고 한다. 필요한 모든 데이터가 3개의 데이터셋에 포함되어 있지만, 결과를 얻으려면 데이터셋을 결합해야 한다.

먼저 인구(`pop`) 데이터프레임 내에서 전체 주 이름을 제공하기 위해 약어(`abbrevs`) 데이터셋과 다대일 병합을 한다. `pop`의 `state/region` 열과 `abbrevs`의 `abbreviation` 열을 기준으로 병합한다. 일치하지 않는 레이블로 인해 데이터가 버려지지 않도록 how='outer'를 사용한다.

In [ ]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', axis=1) # drop duplicate column
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


여기에 불일치하는 항목이 있는지 다시 확인해 보자. `null`이 있는 행을 찾으면 된다.

In [ ]:
merged.isna()

,state/region,ages,year,population,state
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
2539,False,False,False,False,True
2540,False,False,False,False,True
2541,False,False,False,False,True
2542,False,False,False,False,True


In [ ]:
merged.isnull().any() # any() returns True if at least one element within a series or along a Dataframe axis is True

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

`population` 정보 중 일부가 `null`이다. 어떤 정보인지 알아보자!



In [ ]:
merged[merged['population'].isnull()] # isnull() is an alias of isna()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN
2453,PR,under18,1993,NaN,NaN
2454,PR,under18,1992,NaN,NaN
2455,PR,total,1992,NaN,NaN
2456,PR,under18,1994,NaN,NaN
2457,PR,total,1994,NaN,NaN


`population` 값이 `null`인 경우는 2000년 이전의 푸에르토리코(`PR`)에서 나온 것으로 보이는데, 이는 원래 소스에서 이 데이터를 사용할 수 없기 때문일 가능성이 높다.

더 중요한 것은 `state` 항목 중 일부도 `null`이며, 이는 `abbrevs` 키에 해당하는 항목이 없다는 것을 의미한다. 어떤 지역에서 이 일치 항목이 없는지 알아보자:

In [ ]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

`population` 데이터에 푸에르토리코(PR) 및 미국 전체(USA)에 대한 항목이 포함되어 있지만 이러한 항목이 주 약어 키에 나타나지 않는 것을 알 수 있다. 적절한 항목을 채우면 이 문제를 빠르게 해결할 수 있다:

In [ ]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

`state` 열에 더 이상 `null`이 없다. 이제 모든 준비가 완료되었다!

이제 비슷한 절차를 사용하여 결과를 `area` 데이터와 병합할 수 있다.
결과를 살펴보면 두 데이터 모두에서 ``state`` 열을 조인하고 싶을 것이다:

In [ ]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [ ]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

'area' 열에 `null`이 있으므로 어떤 지역인지 살펴보자:

In [ ]:
final.loc[final['area (sq. mi)'].isnull(), 'state'].unique()

array(['United States'], dtype=object)

`areas` 데이터 프레임에 미국 전체의 면적이 포함되어 있지 않다는 것을 알 수 있다. 적절한 값(예: 모든 주 면적의 합계 사용)을 삽입할 수 있지만, 이 경우 미국 전체의 인구 밀도는 현재 논의와 관련이 없으므로 `null` 값을 삭제하자:

In [ ]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


이제 필요한 모든 데이터를 확보했다. 관심 있는 질문에 답하기 위해 먼저 데이터에서 2000년에 해당하는 부분과 총 인구를 선택해 보겠다.

In [ ]:
data2010 = final.loc[(final["year"] == 2010) & (final['ages'] == 'total')]
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


이제 인구 밀도를 계산하여 순서대로 표시해 보자.
먼저 주에 대한 데이터를 다시 색인화한(re-indexing) 다음 결과를 계산한다:

In [ ]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']
density

state
Alabama                   91.287603
Alaska                     1.087509
Arizona                   56.214497
Arkansas                  54.948667
California               228.051342
Colorado                  48.493718
Connecticut              645.600649
Delaware                 460.445752
District of Columbia    8898.897059
Florida                  286.597129
Georgia                  163.409902
Hawaii                   124.746707
Idaho                     18.794338
Illinois                 221.687472
Indiana                  178.197831
Iowa                      54.202751
Kansas                    34.745266
Kentucky                 107.586994
Louisiana                 87.676099
Maine                     37.509990
Maryland                 466.445797
Massachusetts            621.815538
Michigan                 102.015794
Minnesota                 61.078373
Mississippi               61.321530
Missouri                  86.015622
Montana                    6.736171
Nebraska              

In [ ]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [ ]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

이러한 유형의 지저분한 데이터 병합은 실제 데이터 원본을 사용하여 질문에 답하려고 할 때 흔히 하는 작업이다.
이 예제를 통해 데이터에서 인사이트를 얻기 위해 지금까지 다룬 도구를 결합할 수 있는 방법에 대한 아이디어를 얻었기를 바란다.